In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [2]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [3]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

In [4]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [5]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 76874.5156 Validation Accuracy: 0.164062
Epoch  1, Batch   2 -Loss: 61054.5547 Validation Accuracy: 0.234375
Epoch  1, Batch   3 -Loss: 63073.8203 Validation Accuracy: 0.257812
Epoch  1, Batch   4 -Loss: 48355.0781 Validation Accuracy: 0.265625
Epoch  1, Batch   5 -Loss: 47918.5391 Validation Accuracy: 0.257812
Epoch  1, Batch   6 -Loss: 38867.5664 Validation Accuracy: 0.312500
Epoch  1, Batch   7 -Loss: 31016.6719 Validation Accuracy: 0.300781
Epoch  1, Batch   8 -Loss: 34125.6953 Validation Accuracy: 0.292969
Epoch  1, Batch   9 -Loss: 28473.1484 Validation Accuracy: 0.296875
Epoch  1, Batch  10 -Loss: 30704.9766 Validation Accuracy: 0.312500
Epoch  1, Batch  11 -Loss: 26862.9414 Validation Accuracy: 0.320312
Epoch  1, Batch  12 -Loss: 20984.7988 Validation Accuracy: 0.316406
Epoch  1, Batch  13 -Loss: 21801.5918 Validation Accuracy: 0.332031
Epoch  1, Batch  14 -Loss: 19302.8281 Validation Accuracy: 0.328125
Epoch  1, Batch  15 -Loss: 14586.6055 Validation

Epoch  1, Batch 122 -Loss:  3804.3257 Validation Accuracy: 0.718750
Epoch  1, Batch 123 -Loss:  3421.5850 Validation Accuracy: 0.722656
Epoch  1, Batch 124 -Loss:  3361.3809 Validation Accuracy: 0.734375
Epoch  1, Batch 125 -Loss:  3888.4810 Validation Accuracy: 0.714844
Epoch  1, Batch 126 -Loss:  3512.4536 Validation Accuracy: 0.710938
Epoch  1, Batch 127 -Loss:  3484.7534 Validation Accuracy: 0.730469
Epoch  1, Batch 128 -Loss:  4136.5605 Validation Accuracy: 0.730469
Epoch  1, Batch 129 -Loss:  2697.3289 Validation Accuracy: 0.718750
Epoch  1, Batch 130 -Loss:  2530.5684 Validation Accuracy: 0.726562
Epoch  1, Batch 131 -Loss:  3622.4885 Validation Accuracy: 0.722656
Epoch  1, Batch 132 -Loss:  2891.7690 Validation Accuracy: 0.726562
Epoch  1, Batch 133 -Loss:  4106.5679 Validation Accuracy: 0.730469
Epoch  1, Batch 134 -Loss:  2901.2192 Validation Accuracy: 0.734375
Epoch  1, Batch 135 -Loss:  4091.8225 Validation Accuracy: 0.730469
Epoch  1, Batch 136 -Loss:  4166.0503 Validation

Epoch  1, Batch 243 -Loss:  2591.2734 Validation Accuracy: 0.785156
Epoch  1, Batch 244 -Loss:  2127.8267 Validation Accuracy: 0.792969
Epoch  1, Batch 245 -Loss:  1744.3740 Validation Accuracy: 0.785156
Epoch  1, Batch 246 -Loss:  3280.4148 Validation Accuracy: 0.789062
Epoch  1, Batch 247 -Loss:  2378.1758 Validation Accuracy: 0.789062
Epoch  1, Batch 248 -Loss:  1603.2352 Validation Accuracy: 0.785156
Epoch  1, Batch 249 -Loss:  3170.1270 Validation Accuracy: 0.785156
Epoch  1, Batch 250 -Loss:  2075.8662 Validation Accuracy: 0.789062
Epoch  1, Batch 251 -Loss:  2137.3237 Validation Accuracy: 0.789062
Epoch  1, Batch 252 -Loss:  1873.4380 Validation Accuracy: 0.785156
Epoch  1, Batch 253 -Loss:  2199.8823 Validation Accuracy: 0.781250
Epoch  1, Batch 254 -Loss:  1979.9379 Validation Accuracy: 0.781250
Epoch  1, Batch 255 -Loss:  1748.2139 Validation Accuracy: 0.789062
Epoch  1, Batch 256 -Loss:  2741.8364 Validation Accuracy: 0.796875
Epoch  1, Batch 257 -Loss:  2875.1914 Validation

Epoch  1, Batch 364 -Loss:  2696.5664 Validation Accuracy: 0.812500
Epoch  1, Batch 365 -Loss:  1508.1416 Validation Accuracy: 0.808594
Epoch  1, Batch 366 -Loss:  2014.2422 Validation Accuracy: 0.808594
Epoch  1, Batch 367 -Loss:  1426.5566 Validation Accuracy: 0.812500
Epoch  1, Batch 368 -Loss:  1368.3802 Validation Accuracy: 0.800781
Epoch  1, Batch 369 -Loss:  1841.1035 Validation Accuracy: 0.808594
Epoch  1, Batch 370 -Loss:  2100.7620 Validation Accuracy: 0.808594
Epoch  1, Batch 371 -Loss:  1371.1392 Validation Accuracy: 0.808594
Epoch  1, Batch 372 -Loss:  1949.6858 Validation Accuracy: 0.824219
Epoch  1, Batch 373 -Loss:  1599.7444 Validation Accuracy: 0.816406
Epoch  1, Batch 374 -Loss:  1526.1185 Validation Accuracy: 0.816406
Epoch  1, Batch 375 -Loss:  1477.4186 Validation Accuracy: 0.812500
Epoch  1, Batch 376 -Loss:  1433.1799 Validation Accuracy: 0.808594
Epoch  1, Batch 377 -Loss:  1737.0428 Validation Accuracy: 0.808594
Epoch  1, Batch 378 -Loss:  1625.2467 Validation

Epoch  2, Batch  56 -Loss:  1466.4963 Validation Accuracy: 0.812500
Epoch  2, Batch  57 -Loss:  1457.3440 Validation Accuracy: 0.820312
Epoch  2, Batch  58 -Loss:  1640.0144 Validation Accuracy: 0.816406
Epoch  2, Batch  59 -Loss:  1890.0049 Validation Accuracy: 0.816406
Epoch  2, Batch  60 -Loss:  1615.9109 Validation Accuracy: 0.816406
Epoch  2, Batch  61 -Loss:  1370.1434 Validation Accuracy: 0.820312
Epoch  2, Batch  62 -Loss:  1576.5663 Validation Accuracy: 0.812500
Epoch  2, Batch  63 -Loss:  1296.4194 Validation Accuracy: 0.816406
Epoch  2, Batch  64 -Loss:  1323.1462 Validation Accuracy: 0.824219
Epoch  2, Batch  65 -Loss:  1671.3245 Validation Accuracy: 0.816406
Epoch  2, Batch  66 -Loss:  1173.9302 Validation Accuracy: 0.824219
Epoch  2, Batch  67 -Loss:  2213.1614 Validation Accuracy: 0.824219
Epoch  2, Batch  68 -Loss:   933.8967 Validation Accuracy: 0.820312
Epoch  2, Batch  69 -Loss:  1332.9508 Validation Accuracy: 0.824219
Epoch  2, Batch  70 -Loss:  1642.5908 Validation

...

Epoch  9, Batch 320 -Loss:   225.6177 Validation Accuracy: 0.871094
Epoch  9, Batch 321 -Loss:   213.0845 Validation Accuracy: 0.875000
Epoch  9, Batch 322 -Loss:   297.2420 Validation Accuracy: 0.875000
Epoch  9, Batch 323 -Loss:   170.9354 Validation Accuracy: 0.875000
Epoch  9, Batch 324 -Loss:   224.2689 Validation Accuracy: 0.871094
Epoch  9, Batch 325 -Loss:   290.4102 Validation Accuracy: 0.867188
Epoch  9, Batch 326 -Loss:   446.0640 Validation Accuracy: 0.871094
Epoch  9, Batch 327 -Loss:   299.7640 Validation Accuracy: 0.878906
Epoch  9, Batch 328 -Loss:   146.6185 Validation Accuracy: 0.871094
Epoch  9, Batch 329 -Loss:   167.7399 Validation Accuracy: 0.875000
Epoch  9, Batch 330 -Loss:   290.3989 Validation Accuracy: 0.871094
Epoch  9, Batch 331 -Loss:   493.9125 Validation Accuracy: 0.875000
Epoch  9, Batch 332 -Loss:   259.9824 Validation Accuracy: 0.871094
Epoch  9, Batch 333 -Loss:   339.7785 Validation Accuracy: 0.871094
Epoch  9, Batch 334 -Loss:   190.1246 Validation

Epoch 10, Batch  12 -Loss:   360.0096 Validation Accuracy: 0.871094
Epoch 10, Batch  13 -Loss:   350.5199 Validation Accuracy: 0.871094
Epoch 10, Batch  14 -Loss:   221.8403 Validation Accuracy: 0.871094
Epoch 10, Batch  15 -Loss:   149.1286 Validation Accuracy: 0.871094
Epoch 10, Batch  16 -Loss:   216.6292 Validation Accuracy: 0.871094
Epoch 10, Batch  17 -Loss:   294.4624 Validation Accuracy: 0.871094
Epoch 10, Batch  18 -Loss:   274.9908 Validation Accuracy: 0.871094
Epoch 10, Batch  19 -Loss:   457.8148 Validation Accuracy: 0.871094
Epoch 10, Batch  20 -Loss:   309.6287 Validation Accuracy: 0.871094
Epoch 10, Batch  21 -Loss:   343.1340 Validation Accuracy: 0.875000
Epoch 10, Batch  22 -Loss:   265.5643 Validation Accuracy: 0.875000
Epoch 10, Batch  23 -Loss:   199.0462 Validation Accuracy: 0.875000
Epoch 10, Batch  24 -Loss:   390.8568 Validation Accuracy: 0.875000
Epoch 10, Batch  25 -Loss:   398.1809 Validation Accuracy: 0.871094
Epoch 10, Batch  26 -Loss:   408.2940 Validation

Epoch 10, Batch 133 -Loss:   446.0805 Validation Accuracy: 0.878906
Epoch 10, Batch 134 -Loss:    71.4349 Validation Accuracy: 0.875000
Epoch 10, Batch 135 -Loss:   331.2617 Validation Accuracy: 0.875000
Epoch 10, Batch 136 -Loss:   352.9748 Validation Accuracy: 0.875000
Epoch 10, Batch 137 -Loss:   255.3692 Validation Accuracy: 0.875000
Epoch 10, Batch 138 -Loss:   217.9314 Validation Accuracy: 0.875000
Epoch 10, Batch 139 -Loss:   312.1503 Validation Accuracy: 0.871094
Epoch 10, Batch 140 -Loss:   290.2207 Validation Accuracy: 0.871094
Epoch 10, Batch 141 -Loss:   359.3940 Validation Accuracy: 0.871094
Epoch 10, Batch 142 -Loss:   294.6038 Validation Accuracy: 0.875000
Epoch 10, Batch 143 -Loss:   410.5952 Validation Accuracy: 0.875000
Epoch 10, Batch 144 -Loss:   407.3563 Validation Accuracy: 0.875000
Epoch 10, Batch 145 -Loss:   374.6485 Validation Accuracy: 0.875000
Epoch 10, Batch 146 -Loss:   166.3714 Validation Accuracy: 0.875000
Epoch 10, Batch 147 -Loss:   294.4609 Validation

Epoch 10, Batch 254 -Loss:   201.0498 Validation Accuracy: 0.875000
Epoch 10, Batch 255 -Loss:   302.9738 Validation Accuracy: 0.875000
Epoch 10, Batch 256 -Loss:   326.7332 Validation Accuracy: 0.871094
Epoch 10, Batch 257 -Loss:   255.1741 Validation Accuracy: 0.871094
Epoch 10, Batch 258 -Loss:   209.3572 Validation Accuracy: 0.871094
Epoch 10, Batch 259 -Loss:   406.6165 Validation Accuracy: 0.871094
Epoch 10, Batch 260 -Loss:   262.3868 Validation Accuracy: 0.871094
Epoch 10, Batch 261 -Loss:   289.8328 Validation Accuracy: 0.871094
Epoch 10, Batch 262 -Loss:   271.6418 Validation Accuracy: 0.871094
Epoch 10, Batch 263 -Loss:   363.9911 Validation Accuracy: 0.875000
Epoch 10, Batch 264 -Loss:   252.9075 Validation Accuracy: 0.875000
Epoch 10, Batch 265 -Loss:   197.5899 Validation Accuracy: 0.875000
Epoch 10, Batch 266 -Loss:   118.3790 Validation Accuracy: 0.875000
Epoch 10, Batch 267 -Loss:   281.1302 Validation Accuracy: 0.878906
Epoch 10, Batch 268 -Loss:   277.7481 Validation

Epoch 10, Batch 375 -Loss:   404.5305 Validation Accuracy: 0.878906
Epoch 10, Batch 376 -Loss:   346.7352 Validation Accuracy: 0.875000
Epoch 10, Batch 377 -Loss:   356.1095 Validation Accuracy: 0.875000
Epoch 10, Batch 378 -Loss:   408.4865 Validation Accuracy: 0.875000
Epoch 10, Batch 379 -Loss:   255.2489 Validation Accuracy: 0.875000
Epoch 10, Batch 380 -Loss:   329.4408 Validation Accuracy: 0.875000
Epoch 10, Batch 381 -Loss:   233.5277 Validation Accuracy: 0.878906
Epoch 10, Batch 382 -Loss:   309.5638 Validation Accuracy: 0.875000
Epoch 10, Batch 383 -Loss:   503.9225 Validation Accuracy: 0.875000
Epoch 10, Batch 384 -Loss:   220.3048 Validation Accuracy: 0.875000
Epoch 10, Batch 385 -Loss:   286.3544 Validation Accuracy: 0.878906
Epoch 10, Batch 386 -Loss:   318.9436 Validation Accuracy: 0.878906
Epoch 10, Batch 387 -Loss:   301.6025 Validation Accuracy: 0.878906
Epoch 10, Batch 388 -Loss:   228.6774 Validation Accuracy: 0.878906
Epoch 10, Batch 389 -Loss:   221.4168 Validation